In [14]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [15]:
import cv2
import numpy as np
import math
import glob
import csv
import os
import time
#import matplotlib.pyplot as plt
from skimage.exposure import histogram
#from matplotlib.pyplot import bar
from skimage.feature import local_binary_pattern
from skimage import io ,filters
from sklearn.svm import SVC
from mpl_toolkits.mplot3d import Axes3D

In [16]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        plt.axis('off')
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

In [17]:
def Preprocess(image):
     
    # Remove salt and pepper noise    
    # Remove noise 
    median = cv2.medianBlur(image,5)
    blur = cv2.GaussianBlur(image,(5,5),0)
   
    greyImg = image
    
    # Otsu's Binarization
    ret3,img = cv2.threshold(image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
   
    # Remove header and footer
    length, width = img.shape
    up, down, left, right = 0, length - 1, 0, width - 1

    minWidthOfLines = width/2
    contours,__ = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    for i in contours:
        x, y, w, h = cv2.boundingRect(i)
        if w < minWidthOfLines:
            continue
        if y < length // 2:
            up = max(up, y + 15)
        else:
            down = min(down, y - 15)

    offset = 30
    left = left + offset
    right = right -offset
    noHeaderAndFooter = img[up:down + 1, left:right + 1]
    noHeaderAndFooter = np.asarray(noHeaderAndFooter)
    
    noHeaderAndFooterGrey = greyImg[up:down + 1, left:right + 1]
    noHeaderAndFooterGrey = np.asarray(noHeaderAndFooterGrey) 
   
    # To crop the image
    row, col = noHeaderAndFooter.shape
    tolerance = 5

    sumOfRows = np.sum(noHeaderAndFooter, axis = 1)
    rowIndices = np.where(sumOfRows< (col-tolerance)*255)
    up = np.min(rowIndices)
    down = np.max(rowIndices)

    sumOfColoumns = np.sum(noHeaderAndFooter, axis = 0)
    colIndices = np.where(sumOfColoumns< (row-tolerance)*255)
    left = np.min(colIndices)
    right = np.max(colIndices)

    binarized = noHeaderAndFooter[up:down + 1, left:right + 1]
    binarized = np.asarray(binarized)
    
    greyscale = noHeaderAndFooterGrey[up:down + 1, left:right + 1]
    greyscale = np.asarray(greyscale)
   
    # Segmentation of Lines
    rowIndicesShifted = np.roll(rowIndices, -1)
    rowIndicesShifted = rowIndicesShifted[0]

    transitionIndices = np.where(np.abs(rowIndices - rowIndicesShifted) > 10)
    transitionIndices = transitionIndices[1]

    rowIndices = rowIndices[0]

    downIndices= rowIndices[transitionIndices]

    transitionIndicesUp = np.insert(transitionIndices,0,-1)
    transitionIndicesUp = np.delete(transitionIndicesUp,-1)

    upIndices= rowIndices[transitionIndicesUp+1]
    
    segmentsBinarized = []
    segmentsGrey = []
    totalSize=0
    whiteSpaceTolerance=0.975
    for i in range(transitionIndices.shape[0]):
        currSegment= noHeaderAndFooterGrey[upIndices[i]:downIndices[i] + 1, left:right + 1]
        if((np.sum(currSegment))<(currSegment.shape[0]*currSegment.shape[1]*whiteSpaceTolerance*255)):
            segmentsBinarized.append(noHeaderAndFooter[upIndices[i]:downIndices[i] + 1, left:right + 1])
            segmentsGrey.append(currSegment)
          
    segmentsGrey=np.asarray(segmentsGrey)
    
    return greyscale, binarized, segmentsBinarized, segmentsGrey

In [18]:
def showHist(imgHist):
    bar(imgHist[1].astype(np.uint8), imgHist[0], width=0.8, align='center')
    return imgHist[0]

In [19]:
#  Feature Extraction LBP
def LBP(greyscale):
    lbp = local_binary_pattern(greyscale, 8, 3, method='uniform')
    n_bins =10 #256 
    imgHist = histogram(lbp, n_bins)
    
    return imgHist[0];

In [20]:
def extract_features(greyscale1, binarized1, segmentsBinarized1, segmentsGrey1):
    return LBP(segmentsGrey1)

In [21]:
def train(folder):
    trainTime=0
    feature_vector_all=[]
    direcs = glob.glob (folder+"/*")
    for direc in direcs:
        if os.path.isdir(direc):
            print(direc)
            files = glob.glob (direc+'/*')
            for file in files:
                #reading an imag
                print("-----------------------",file,"------------------------------")
                img = cv2.imread(file,0) 
                tempStart =time.time()

                #converting it to a gray image
                #binarization process
                
                greyscale1, binarized1, segmentsBinarized1, segmentsGrey1 = Preprocess(img)
                print(segmentsGrey1.shape)
                for i in range(segmentsGrey1.shape[0]):
                    feature_vector=[]
                    
                    feature_vector.append(int(direc[8]))
                    
                    hist1 = extract_features(greyscale1, binarized1, segmentsBinarized1, segmentsGrey1[i])
                    feature_vector.extend(hist1)
                    feature_vector_all.append(feature_vector)
                tempEnd =time.time()
                trainTime+= tempEnd-tempStart 

    return feature_vector_all,trainTime

In [22]:
def calculateDistance(x1, x2):

    distance =np.linalg.norm(x1-x2)
    return distance

# def KNN(test_point, training_features, labels, k): 
#     y= labels[ np.argsort(calculateDistance(test_point,training_features))[:k]]
#     l,h = np.unique(y, return_counts=True)
#     return l[np.argmax(h)]

#     distarr=[]
#     for i in range (training_features.shape[0]):
#         f=calculateDistance(training_features[i,:],test_point)
#         distarr.append(f)
        
        
#     sortedarr=np.sort(distarr)
    
#     classes=np.zeros(3)
    
#     for i in range(k):
#         result = np.where(distarr == sortedarr[i])
#         print(result)
#         for j in range(3):
#             if(labels[result[0]]==j+1):
#                 classes[j]+=1
            
            
#     classification=np.argmax(classes)

  
#     return classification+1




In [23]:
def KNN(test_point, training_features, y_train, k):
    class1=0
    class2=0
    class3=0
    dist=[]
    indexs=[]
    
    for i in range(training_features.shape[0]):
        dist.append(calculateDistance(test_point,training_features[i]))
        
    dist2=np.argsort(dist)

    for i in range(k):
        if(y_train[dist2[i]]==1):
            class1=class1+1
        elif(y_train[dist2[i]]==2):
            class2=class2+1
        else:
            class3=class3+1

    if(max(class1,class2,class3)==class1):
        classification=1
    elif(max(class1,class2,class3)==class2):
        classification=2
    else:
        classification=3
    return classification

In [24]:
def SVM(training_features, y_train):
    clf = SVC(kernel='poly')    #linear sigmoid
    clf.fit(training_features, y_train)  
    return clf

In [25]:
def main(directory,true_values,l):
    training_data,trainTime = train(directory)

    training_data = np.asarray(training_data)

    labels = training_data[:,0]
    training_features = training_data[:,1:]
    test_images = sorted(glob.glob(directory+'/*.png'))

    ids = open('ids.txt')
    ids = ids.readlines()
    ids=np.asarray(ids,dtype=int)
    xtest=ids[-1]
    ytest=np.where(ids==xtest)[0][0]+1
    #print(ytest)
    
    true_values.append(ytest)
    print(true_values[l])

    #true_values = int(str(os.path.basename(test_images[0])).split(".")[0])
    k = 5
    knn_prediction = []
    SVM_prediction=[]
    totalTime=[]

    f = open("time.txt", "a")
    f2= open("results.txt","a")
    clf= SVM(training_features,labels)

    for i in range(len(test_images)):
    
        img_original = cv2.imread(test_images[i],0)
        start = time.time()
        greyscale1, binarized1, segmentsBinarized1, segmentsGrey1 = Preprocess(img_original)
        maxOccurSegment=[]
        maxOccurSegmentSVM=[]
        print("Actual class :", true_values, " of file ", test_images[i])
        print("---------------------------------------")
        for j in range(segmentsGrey1.shape[0]):
            test_point = extract_features(greyscale1, binarized1, segmentsBinarized1, segmentsGrey1[j])

            maxOccurSegment.append(KNN(test_point, training_features, labels, k))
            maxOccurSegmentSVM.append(clf.predict([test_point])[0])  #SVM


        maxOccurSegment=np.asarray(maxOccurSegment)
        maxOccurSegmentSVM=np.asarray(maxOccurSegmentSVM)

        print(maxOccurSegmentSVM)
        knn_prediction.append(np.bincount(maxOccurSegment).argmax())
        SVM_prediction.append(np.bincount(maxOccurSegmentSVM).argmax())
        end = time.time()
        predictionTime= end-start
        totalTime.append(trainTime+ predictionTime)

        f.write(str(round(totalTime[i],2)))
        f.write("\n")

        f2.write(str(knn_prediction[i]))
        f2.write("\n")


        
        print("knn_prediction class :", knn_prediction[i])
        print("SVM_prediction class :", SVM_prediction[i])
        # ------------------------------------------------------------------------------------------------------

    f.write("------------------------------------")
    totalTime=np.asarray(totalTime)
    f.write(str(np.mean(totalTime)))
    f.close()
    f2.close()

    
    return True

In [26]:
direcs = glob.glob ("data/*")
true_values = []
knn_prediction = []
SVM_prediction=[]

for l in range(len(direcs)):
    knn_predict, SVM_predict = main(direcs[l],true_values,l)
    knn_prediction.append(knn_predict[0])
    SVM_prediction.append(SVM_predict[0])

data\00\1
----------------------- data\00\1\b06-042.png ------------------------------


ValueError: too many values to unpack (expected 2)

In [1048]:
total_predictions =  len(direcs)

correct_knn=0
correct_SVM=0
for i in range(total_predictions):
    if(true_values[i]==knn_prediction[i]):
        correct_knn=correct_knn+1
    if(true_values[i]==SVM_prediction[i]):
        correct_SVM=correct_SVM+1

accuracy_knn = (correct_knn/total_predictions)*100
accuracy_SVM = (correct_SVM/total_predictions)*100
# ------------------------------------------------------------------------------------------------------

print("K-Nearest Neighbour Classifier Accuracy: ", accuracy_knn, "%")
print("SVM Classifier Accuracy: ", accuracy_SVM, "%")

K-Nearest Neighbour Classifier Accuracy:  100.0 %
SVM Classifier Accuracy:  100.0 %
